In [1]:
import pandas as pd
import numpy as np
import json
import sys
import folium
import branca.colormap as cm
import os
import time
import datetime
from selenium import webdriver

sys.path.append('..')

In [2]:
def getColor(d):
    try:
        return lin(rate_dict[d])
    except:
        return '#FFFFFF'

In [3]:
df = pd.read_csv('data/rt_datewise.csv')
df1 = df[['date', 'state', 'ML']]
dates = df1.date.unique()

In [4]:
dates.shape[0]

44

In [5]:
index  = 2
dates[index]
datetime.datetime.strptime(dates[index], '%Y-%m-%d').strftime('%d-%b-%Y')

'16-Mar-2020'

In [ ]:
for index in range(dates.shape[0]):
    print(dates[index])
    df2 = df1[df1['date']==dates[index]]
    df2 = df2.replace(['AP', 'BR', 'DL', 'GJ', 'HR', 'JK', 'KA', 'KL', 'MH', 'MP', 'PB',
       'RJ', 'TG', 'TN', 'UP', 'WB'],
        ['Andhra Pradesh',
        'Bihar',
        'Delhi',
        'Gujarat',
        'Haryana',
        'Jammu & Kashmir',
        'Karnataka',
        'Kerala', 'Maharashtra',                                    
        'Madhya Pradesh',
        'Punjab',
        'Rajasthan',
        'Telengana',  
        'Tamil Nadu',
        'Uttar Pradesh',
        'West Bengal'])

    geo_json_data = json.load(open('data/india/state/india_telengana.geojson'))
    rate_dict = df2.set_index('state')['ML']
    added_list = ['Andhra Pradesh',
                    'Bihar',
                    'Delhi',
                    'Gujarat',
                    'Haryana',
                    'Jammu & Kashmir',
                    'Karnataka',
                    'Kerala', 
                    'Maharashtra',                                    
                    'Madhya Pradesh',
                    'Punjab',
                    'Rajasthan',
                    'Telengana',  
                    'Tamil Nadu',
                    'Uttar Pradesh',
                    'West Bengal']

    i = 0
    for key in geo_json_data['features']:
        if geo_json_data['features'][i]['properties']['NAME_1'] not in added_list:
            st = geo_json_data['features'][i]['properties']['NAME_1']
            rate_dict[st] = 0
            i = i+1

    m = folium.Map(location=[21, 78], tiles='cartodbpositron', zoom_start=4)

    lin = cm.LinearColormap(['#004d00','#00cc00','#ffd9b3','#ff6966',
                            '#e60400','#ff0400', '#660000'], 
                            index = [0,0.5,0.7,0.9,1.2,1.5,3.5], vmin=0, vmax=6)    

    folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': getColor(feature['properties']['NAME_1']),
        'color' : 'black',
        'weight' : 2,
        'dashArray' : '5, 5'
        }
    ).add_to(m)

    title_html = '''
                 <h3 align="center" style="font-size:20px"><b>State wise Real-time R0(Rt) for India as on {}</b></h3>
                 '''.format(datetime.datetime.strptime(dates[index], '%Y-%m-%d').strftime('%d-%b-%Y'))
    m.get_root().html.add_child(folium.Element(title_html))

    m
    print('Map generated')

    m.save('{}.html'.format(dates[index]))
    print('Map saved')

    delay=5
    time.sleep(3)

    fn='{}.html'.format(dates[index])
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    m.save(fn)

    browser = webdriver.Chrome('chromedriver')
    browser.get(tmpurl)
    time.sleep(delay)
    browser.save_screenshot('Rt_{}.png'.format(dates[index]))
    browser.quit()
    print('Rt_{}.png Image saved'.format(dates[index]))

2020-03-14
Map generated
Map saved


In [ ]:
# Converting to GIF
import imageio
with imageio.get_writer('Rt_India.gif', mode = 'I') as writer:
    for index in range(dates.shape[0]):
        image = imageio.imread('Rt_{}.png'.format(dates[index]))
        writer.append_data(image)


In [43]:
# Converting to GIF
import imageio

exportname = "Rt_India.gif"
kargs = { 'duration': 0.5 }
frames = []        
for index in range(dates.shape[0]):
    image = imageio.imread('Rt_{}.png'.format(dates[index]))
    frames.append(image)

imageio.mimsave(exportname, frames, 'GIF', **kargs)

In [44]:
!convert -loop 1 Rt_India.gif Rt_India_single.gif